In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import os

In [2]:
df_ = pd.read_csv('presidential_speeches.csv', sep=',', usecols=['Party', 'Transcript'], encoding='utf-8')

df_dems = df_[df_['Party'] == 'Democratic']
df_rebs = df_[df_['Party'] == 'Republican']
df = pd.concat([df_dems, df_rebs])
df['Party'] = df['Party'].apply(lambda x: 1 if x == 'Republican' else 0)
df.sample(10)

,Party,Transcript
941,0,"Mr. President, Mr. Secretary-General, my fello..."
257,0,To the House of Representatives: I return with...
458,1,"Mr. Chairman *, the message which you have for..."
119,0,Fellow Citizens: The practice of all my predec...
997,0,"THE PRESIDENT: Well, thank you. Please, if you..."
268,1,To the Senate of the United States: I transmit...
439,0,"Madam President, Ladies of the Association: I ..."
680,0,"THE PRESIDENT. Good afternoon, ladies and gent..."
758,1,Good evening. I am here tonight to announce my...
984,1,"Thank you to our First Lady, Melania, who has ..."


In [3]:
df.isnull().sum()

Party         0
Transcript    0
dtype: int64

In [3]:
X = df['Transcript'].values
y = df['Party'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((702,), (176,), (702,), (176,))

In [14]:
print(X_train[0])

President Anderson, members of the faculty, board of trustees, distinguished guests, my old colleague, Senator Bob Byrd, who has earned his degree through many years of attending night law school while I am earning mine in the next 30 minutes, ladies and gentlemen: It is with great pride that I participate in this ceremony of the American University, sponsored by the Methodist Church, founded by Bishop John Fletcher Hurst, and first opened by President Woodrow Wilson in 1914. This is a young and growing university, but it has already fulfilled Bishop Hurst's enlightened hope for the study of history and public affairs in a city devoted to the making of history and to the conduct of the public's business. By sponsoring this institution of higher learning for all who wish to learn, whatever their color or their creed, the Methodists of this area and the Nation deserve the Nation's thanks, and I commend all those who are today graduating. Professor Woodrow Wilson once said that every man 

In [16]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((702,), (176,), (702,), (176,))

In [19]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_ = tokenizer.texts_to_sequences(X_train)
X_test_ = tokenizer.texts_to_sequences(X_test)

voc_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(X_train[2])
print(X_train_[2])

Fellow Citizens of the Senate and House of Representatives: The members of the Forty-sixth Congress have assembled in their first regular session under circumstances calling for mutual congratulation and grateful acknowledgment to the Giver of All Good for the large and unusual measure of national prosperity which we now enjoy. The most interesting events which have occurred in our public affairs since my last annual message to Congress are connected with the financial operations of the Government, directly affecting the business interests of the country. I congratulate Congress on the successful execution of the resumption act. At the time fixed, and in the manner contemplated by law, United States notes began to be redeemed in coin. Since the 1st of January last they have been promptly redeemed on presentation, and in all business transactions, public and private, in all parts of the country, they are received and paid out as the equivalent of coin. The demand upon the Treasury for g

In [20]:
num_words = 5000

def vectorize_sequences(sequences, dimension=num_words): 
    results = np.zeros((len(sequences), dimension)) 
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results 

X_train = vectorize_sequences(X_train_)
X_test = vectorize_sequences(X_test_)

In [6]:
vect = CountVectorizer()
vect.fit(X_train)

CountVectorizer()

In [8]:
vect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [9]:
vect.get_feature_names()[-20:]

['zeros',
 'zest',
 'zigs',
 'zika',
 'zimbabwe',
 'zimbabweans',
 'zimmerman',
 'zinc',
 'zinke',
 'zion',
 'zionism',
 'zip',
 'zone',
 'zones',
 'zoning',
 'zoological',
 'zooming',
 'zoot',
 'zte',
 'zuckert']

In [7]:
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((702, 33651), (176, 33651), (702,), (176,))

## Model 1: Dense Network

In [10]:
num_words = X_train.shape[1]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(num_words,)))
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.add(tf.keras.layers.Flatten())
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               6730400   
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
 flatten (Flatten)           (None, 1)                 0         
                                                                 
Total params: 6,750,601
Trainable params: 6,750,601
Non-trainable params: 0
_________________________________________________________________


In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30)

Epoch 1/30


C:\Users\user\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 200), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


22/22 [==============================] - 4s 138ms/step - loss: 8.2263 - accuracy: 0.4459
Epoch 2/30
22/22 [==============================] - 3s 151ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 3/30
22/22 [==============================] - 4s 167ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 4/30
22/22 [==============================] - 4s 165ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 5/30
22/22 [==============================] - 3s 148ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 6/30
22/22 [==============================] - 3s 149ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 7/30
22/22 [==============================] - ETA: 0s - loss: 8.5804 - accuracy: 0.43 - 3s 156ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 8/30
22/22 [==============================] - 4s 161ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 9/30
22/22 [==============================] - 4s 168ms/step - loss: 8.5804 - accuracy: 0.4373
Epoch 10/30
22/22 [==============================] - 4s 169ms/step - 

In [12]:
from sklearn.metrics import classification_report

predictions = [1. if x > 0.5 else 0. for x in model.predict(X_test)]
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.47      0.64       176

    accuracy                           0.47       176
   macro avg       0.50      0.23      0.32       176
weighted avg       1.00      0.47      0.64       176



C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model 2: Simple RNN

In [20]:
from keras.preprocessing import sequence

maxlen = 50

X_train = sequence.pad_sequences(X_train.toarray(), maxlen=maxlen, truncating="pre", padding="post")
X_test = sequence.pad_sequences(X_test.toarray(), maxlen=maxlen, truncating="pre", padding="post")

In [21]:
max([len(x) for x in X_test])

50

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(num_words, 32))
model.add(tf.keras.layers.SimpleRNN(32)) # return_sequences=True for getting the complete output sequence
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          1076832   
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,078,945
Trainable params: 1,078,945
Non-trainable params: 0
_________________________________________________________________


In [24]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

num_epochs = 30
history = model.fit(X_train, y_train, epochs=num_epochs)

Epoch 1/30
22/22 [==============================] - 2s 17ms/step - loss: 0.6616 - accuracy: 0.6083
Epoch 2/30
22/22 [==============================] - 0s 23ms/step - loss: 0.6640 - accuracy: 0.6211
Epoch 3/30
22/22 [==============================] - 1s 32ms/step - loss: 0.6591 - accuracy: 0.6026
Epoch 4/30
22/22 [==============================] - 1s 38ms/step - loss: 0.6548 - accuracy: 0.6111
Epoch 5/30
22/22 [==============================] - 1s 39ms/step - loss: 0.6566 - accuracy: 0.6140
Epoch 6/30
22/22 [==============================] - 1s 40ms/step - loss: 0.6586 - accuracy: 0.6140
Epoch 7/30
22/22 [==============================] - 1s 39ms/step - loss: 0.6557 - accuracy: 0.6168
Epoch 8/30
22/22 [==============================] - 1s 41ms/step - loss: 0.6529 - accuracy: 0.6040
Epoch 9/30
22/22 [==============================] - 1s 40ms/step - loss: 0.6534 - accuracy: 0.6211
Epoch 10/30
22/22 [==============================] - 1s 41ms/step - loss: 0.6537 - accuracy: 0.6140
Epoch 11/

In [25]:
predictions = [1. if x > 0.5 else 0. for x in model.predict(X_test)]
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

         0.0       0.98      0.56      0.71       165
         1.0       0.11      0.82      0.19        11

    accuracy                           0.57       176
   macro avg       0.54      0.69      0.45       176
weighted avg       0.92      0.57      0.68       176



## Model 3: Long-Short Term Memory (LSTM)

In [27]:
maxlen = 200

X_train = sequence.pad_sequences(X_train, maxlen=maxlen, truncating="pre", padding="post")
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, truncating="pre", padding="post")

In [28]:
from keras.layers import LSTM

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(num_words, 32))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          1076832   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,085,185
Trainable params: 1,085,185
Non-trainable params: 0
_________________________________________________________________


In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

num_epochs = 30
history = model.fit(X_train, y_train, epochs=num_epochs)

Epoch 1/30
22/22 [==============================] - 8s 139ms/step - loss: 0.6889 - accuracy: 0.5513
Epoch 2/30
22/22 [==============================] - 4s 164ms/step - loss: 0.6875 - accuracy: 0.5627
Epoch 3/30
22/22 [==============================] - 4s 174ms/step - loss: 0.6875 - accuracy: 0.5627
Epoch 4/30
22/22 [==============================] - 3s 135ms/step - loss: 0.6880 - accuracy: 0.5627
Epoch 5/30
22/22 [==============================] - 3s 133ms/step - loss: 0.6859 - accuracy: 0.5627
Epoch 6/30
22/22 [==============================] - 3s 132ms/step - loss: 0.6864 - accuracy: 0.5627
Epoch 7/30
22/22 [==============================] - 3s 133ms/step - loss: 0.6857 - accuracy: 0.5627
Epoch 8/30
22/22 [==============================] - 3s 131ms/step - loss: 0.6862 - accuracy: 0.5627
Epoch 9/30
22/22 [==============================] - 3s 159ms/step - loss: 0.6860 - accuracy: 0.56270s - loss: 0.6885 - ac
Epoch 10/30
22/22 [==============================] - 4s 169ms/step - loss: 0.6

In [30]:
predictions = [1. if x > 0.5 else 0. for x in model.predict(X_test)]
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

         0.0       1.00      0.53      0.70       176
         1.0       0.00      0.00      0.00         0

    accuracy                           0.53       176
   macro avg       0.50      0.27      0.35       176
weighted avg       1.00      0.53      0.70       176



C:\Program Files\Python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
